In [1]:
!pip install --force-reinstall torch torch-geometric faiss-cpu rapidfuzz neo4j scipy

  Using cached torch-2.7.0-cp313-cp313-win_amd64.whl.metadata (29 kB)
  Using cached torch_geometric-2.6.1-py3-none-any.whl.metadata (63 kB)
  Using cached faiss_cpu-1.11.0-cp313-cp313-win_amd64.whl.metadata (5.0 kB)
  Using cached rapidfuzz-3.13.0-cp313-cp313-win_amd64.whl.metadata (12 kB)
  Using cached neo4j-5.28.1-py3-none-any.whl.metadata (5.9 kB)
  Using cached scipy-1.15.3-cp313-cp313-win_amd64.whl.metadata (60 kB)
  Using cached filelock-3.18.0-py3-none-any.whl.metadata (2.9 kB)
  Using cached typing_extensions-4.13.2-py3-none-any.whl.metadata (3.0 kB)
  Using cached sympy-1.14.0-py3-none-any.whl.metadata (12 kB)
  Using cached networkx-3.5-py3-none-any.whl.metadata (6.3 kB)
  Using cached jinja2-3.1.6-py3-none-any.whl.metadata (2.9 kB)
  Using cached fsspec-2025.5.1-py3-none-any.whl.metadata (11 kB)
  Using cached setuptools-80.9.0-py3-none-any.whl.metadata (6.6 kB)
  Using cached aiohttp-3.12.6-cp313-cp313-win_amd64.whl.metadata (7.9 kB)
  Using cached numpy-2.2.6-cp313-cp313

  You can safely remove it manually.


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch_geometric.nn import RGCNConv
import faiss
import numpy as np
import pandas as pd
import json
from torch_geometric.data import Data
from rapidfuzz import process, fuzz

import csv
from neo4j import GraphDatabase

d:\Graph\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:

with open('phone_details.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

# Lựa chọn các trường quan trọng để tạo KG
important_fields = {
    "manufacturer": "Sản xuất bởi",
    "chipset": "Chipset",
    "cpu": "CPU",
    "operating_system": "Hệ điều hành",
    "os_version": "Phiên bản hệ điều hành",
    "display_size": "Kích thước màn hình",
    "display_resolution": "Độ phân giải màn hình",
    "mobile_type_of_display": "Loại màn hình",
    "mobile_tan_so_quet": "Tần số quét",
    "camera_primary": "Camera sau",
    "camera_secondary": "Camera trước",
    "camera_video": "Video ghi hình",
    "mobile_cong_nghe_sac": "Công nghệ sạc",
    "sac_khong_day": "Sạc không dây",
    "mobile_khang_nuoc_bui": "Kháng nước bụi",
    "sim": "Loại sim",
    "storage": "Bộ nhớ",
    "mobile_ram_filter": "RAM",
    "mobile_nfc": "NFC",
    "bluetooth": "Bluetooth",
    "wlan": "Wifi",
    "gps": "GPS",
    "dimensions": "Kích thước",
    "product_weight": "Trọng lượng",
    "mobile_tinh_nang_dac_biet": "Tính năng đặc biệt",
    "included_accessories": "Phụ kiện bao gồm",
    "warranty_information": "Bảo hành"
}

# Tạo danh sách để lưu quan hệ
kg_data = []

# Duyệt qua từng sản phẩm trong mảng
for product in data:
    general = product.get('general', {})
    attributes = general.get('attributes', {})
    product_id = attributes.get('id', {})

    source = general.get('name')
    
    for json_key, rel_name in important_fields.items():
        value = attributes.get(json_key)
        if value and value != "no_selection" and value != "":
            kg_data.append({
                "source": source,
                "relation": rel_name,
                "target": value
            })

# Tạo DataFrame cho KG
kg_df = pd.DataFrame(kg_data)
kg_df.to_csv("triples.csv", index=False, encoding='utf-8')

# Xem thử 10 dòng đầu tiên
print(kg_df.head(10))


                                      source                relation  \
0  iPhone 16 Pro Max 256GB | Chính hãng VN/A            Sản xuất bởi   
1  iPhone 16 Pro Max 256GB | Chính hãng VN/A                 Chipset   
2  iPhone 16 Pro Max 256GB | Chính hãng VN/A                     CPU   
3  iPhone 16 Pro Max 256GB | Chính hãng VN/A            Hệ điều hành   
4  iPhone 16 Pro Max 256GB | Chính hãng VN/A  Phiên bản hệ điều hành   
5  iPhone 16 Pro Max 256GB | Chính hãng VN/A     Kích thước màn hình   
6  iPhone 16 Pro Max 256GB | Chính hãng VN/A   Độ phân giải màn hình   
7  iPhone 16 Pro Max 256GB | Chính hãng VN/A           Loại màn hình   
8  iPhone 16 Pro Max 256GB | Chính hãng VN/A             Tần số quét   
9  iPhone 16 Pro Max 256GB | Chính hãng VN/A              Camera sau   

                                              target  
0                                              Apple  
1                                      Apple A18 Pro  
2  CPU 6 lõi mới với 2 lõi hiệu năng và 4 

In [4]:
# # Thông tin kết nối đến Neo4j
# uri = "bolt://localhost:7687"
# username = "neo4j"
# password = "your_password"  # Thay bằng mật khẩu thực tế của bạn

# # Hàm để chèn triple vào Neo4j
# def insert_triple(tx, head, relation, tail):
#     query = (
#         "MERGE (h:Entity {name: $head}) "
#         "MERGE (t:Entity {name: $tail}) "
#         "MERGE (h)-[r:" + relation + "]->(t)"
#     )
#     tx.run(query, head=head, tail=tail)

# # Kết nối đến Neo4j
# driver = GraphDatabase.driver(uri, auth=(username, password))

# # Đọc tệp CSV và thêm dữ liệu vào Neo4j
# with driver.session() as session:
#     with open('triples.csv', 'r', encoding='utf-8') as file:
#         reader = csv.DictReader(file)
#         for row in reader:
#             head = row['head'].strip()
#             relation = row['relation'].strip()
#             tail = row['tail'].strip()
#             print(f"Tạo triple: ({head})-[:{relation}]->({tail})")
#             session.execute_write(insert_triple, head, relation, tail)

# print("✅ Đã thêm toàn bộ triple vào Neo4j.")
# driver.close()


In [5]:
# Giả sử kg_df có 3 cột: source, relation, target
# Ví dụ: source = 'iPhone 16 Pro Max', relation = 'chipset', target = 'Apple A18 Pro'

# 1. Chuẩn bị entities (từ source và target)
entities = pd.unique(kg_df[['source', 'target']].values.ravel('K'))
entities = list(entities)

# 2. Chuẩn bị relations (từ cột relation)
relations = kg_df['relation'].unique().tolist()

# 3. Tạo dict ánh xạ entity -> id và relation -> id
entity2id = {entity: idx for idx, entity in enumerate(entities)}
relation2id = {rel: idx for idx, rel in enumerate(relations)}
id2entity = {idx: entity for entity, idx in entity2id.items()}

# 4. Tạo list triples (head, relation, tail) theo id hoặc theo tên
# Dạng theo tên:
triples = [(row['source'], row['relation'], row['target']) for idx, row in kg_df.iterrows()]

# Nếu muốn dạng theo id (thường dùng cho mô hình KG embedding):
triples_id = [
    (entity2id[row['source']], relation2id[row['relation']], entity2id[row['target']])
    for idx, row in kg_df.iterrows()
]

# In ra kết quả ví dụ
print("Entities:", entities[:5])
print("Relations:", relations)
print("Triples sample:", triples[:5])
print("Triples (id) sample:", triples_id[:5])


Entities: ['iPhone 16 Pro Max 256GB | Chính hãng VN/A', 'Samsung Galaxy S25 256GB', 'Xiaomi 14T Pro 12GB 512GB', 'OPPO Reno10 Pro+ 5G 12GB 256GB', 'Xiaomi 14T 12GB 512GB']
Relations: ['Sản xuất bởi', 'Chipset', 'CPU', 'Hệ điều hành', 'Phiên bản hệ điều hành', 'Kích thước màn hình', 'Độ phân giải màn hình', 'Loại màn hình', 'Tần số quét', 'Camera sau', 'Camera trước', 'Video ghi hình', 'Công nghệ sạc', 'Sạc không dây', 'Kháng nước bụi', 'Loại sim', 'Bộ nhớ', 'RAM', 'NFC', 'Bluetooth', 'Wifi', 'GPS', 'Kích thước', 'Trọng lượng', 'Tính năng đặc biệt', 'Phụ kiện bao gồm', 'Bảo hành']
Triples sample: [('iPhone 16 Pro Max 256GB | Chính hãng VN/A', 'Sản xuất bởi', 'Apple'), ('iPhone 16 Pro Max 256GB | Chính hãng VN/A', 'Chipset', 'Apple A18 Pro'), ('iPhone 16 Pro Max 256GB | Chính hãng VN/A', 'CPU', 'CPU 6 lõi mới với 2 lõi hiệu năng và 4 lõi hiệu suất'), ('iPhone 16 Pro Max 256GB | Chính hãng VN/A', 'Hệ điều hành', 'iOS'), ('iPhone 16 Pro Max 256GB | Chính hãng VN/A', 'Phiên bản hệ điều hành

In [6]:
triples_idx = [
    (entity2id[head], relation2id[rel], entity2id[tail])
    for head, rel, tail in triples
]

In [7]:
# ======== Mô hình ========
class RGCN(torch.nn.Module):
    def __init__(self, num_entities, num_relations, embedding_dim):
        super(RGCN, self).__init__()
        self.embedding = nn.Embedding(num_entities, embedding_dim)
        self.conv1 = RGCNConv(embedding_dim, embedding_dim, num_relations)
        self.conv2 = RGCNConv(embedding_dim, embedding_dim, num_relations)

    def forward(self, x, edge_index, edge_type):
        x = self.embedding(x)
        x = self.conv1(x, edge_index, edge_type)
        x = torch.relu(x)
        x = self.conv2(x, edge_index, edge_type)
        return x

    def get_score(self, h, r, t):
        # Đơn giản kiểu TransE: -||h + r - t||
        return -torch.norm(h + r - t, dim=1)


In [8]:
# ======== Thông số ========
embedding_dim = 768
num_entities = len(entities)
num_relations = len(relations)

model = RGCN(num_entities, num_relations, embedding_dim)
optimizer = optim.Adam(model.parameters(), lr=0.01)
criterion = nn.MarginRankingLoss(margin=1.0)


In [9]:

edge_index = torch.tensor([[head, tail] for head, _, tail in triples_idx] +
                           [[tail, head] for head, _, tail in triples_idx], dtype=torch.long).t().contiguous()

edge_type = torch.tensor([rel for _, rel, _ in triples_idx] +
                         [rel for _, rel, _ in triples_idx], dtype=torch.long)

x = torch.eye(num_entities, dtype=torch.float)  # Ma trận đơn vị cho embedding ban đầu

data = Data(x=x, edge_index=edge_index, edge_type=edge_type)

In [10]:

# ======== Huấn luyện ========
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# ======== Hàm huấn luyện ========
def train():
    model.train()
    optimizer.zero_grad()

    # Forward để lấy embedding
    x = torch.arange(num_entities)  # node chỉ số
    node_emb = model(x.to(device), data.edge_index.to(device), data.edge_type.to(device))

    # Lấy positive triples
    pos_heads = torch.tensor([entity2id[h] for h, _, _ in triples], device=device)
    pos_rels  = torch.tensor([relation2id[r] for _, r, _ in triples], device=device)
    pos_tails = torch.tensor([entity2id[t] for _, _, t in triples], device=device)

    # Negative sampling: thay tail bằng node ngẫu nhiên
    neg_tails = torch.randint(0, num_entities, pos_tails.shape, device=device)

    h = node_emb[pos_heads]
    t = node_emb[pos_tails]
    t_neg = node_emb[neg_tails]
    r = nn.Embedding(num_relations, embedding_dim).to(device)(pos_rels)

    # Tính score
    pos_score = model.get_score(h, r, t)
    neg_score = model.get_score(h, r, t_neg)

    target = torch.ones_like(pos_score)
    loss = criterion(pos_score, neg_score, target)

    loss.backward()
    optimizer.step()
    return loss.item()


In [11]:

for epoch in range(10):
    loss = train()
    print(f"Epoch {epoch+1}, Loss: {loss:.4f}")


Epoch 1, Loss: 4.2396
Epoch 2, Loss: 308.7006
Epoch 3, Loss: 298.1114
Epoch 4, Loss: 99.6439
Epoch 5, Loss: 73.8271
Epoch 6, Loss: 71.5554
Epoch 7, Loss: 43.1234
Epoch 8, Loss: 28.7193
Epoch 9, Loss: 22.6677
Epoch 10, Loss: 23.9901


In [18]:

# ======== Xuất Embedding và FAISS ========
model.eval()
with torch.no_grad():
    x = torch.arange(num_entities).to(device)
    embeddings = model(x, data.edge_index.to(device), data.edge_type.to(device)).cpu().numpy()

# # FAISS Index
index = faiss.IndexFlatL2(embedding_dim)
index.add(embeddings)
faiss.write_index(index, 'faiss_index.index')

In [19]:

entity_names = [str(key) for key in entity2id.keys()]
entities = ["iPhone 14 128GB  | Chính hãng VN/A", "32 GB"]

for entitie in entities:
    print('Xử lý cho node', entitie)
    # Tìm node giống nhất
    best_match = process.extractOne(entitie, entity_names, scorer=fuzz.WRatio)


    if best_match:
        matched_name, score, idx = best_match
        match_id = entity2id.get(matched_name, "UNKNOWN")
        print(f"🔍 Node giống nhất: {matched_name} (score: {score:.2f}) (id: {match_id})")

        query_embedding = embeddings[match_id]

        # Tải chỉ mục FAISS đã lưu
        index = faiss.read_index('faiss_index.index')

        # Tìm kiếm k nút tương tự nhất
        k = 5
        D, I = index.search(np.array([query_embedding]), k)

        # In ra kết quả
        print("Các nút tương tự nhất:")
        for i in range(k):
            node_idx = I[0][i]
            entity_name = id2entity.get(node_idx, "UNKNOWN")
            print(f"Nút: {entity_name} (id={node_idx})")

            related_triples = [trip for trip in triples if trip[0] == entity_name or trip[2] == entity_name]

            for head, rel, tail in related_triples:
                print(f"    ({head}) -[:{rel}]-> ({tail})")

    else:
        print("❌ Không tìm thấy node tương tự.")


Xử lý cho node iPhone 14 128GB  | Chính hãng VN/A
🔍 Node giống nhất: iPhone 14 128GB  | Chính hãng VN/A (score: 100.00) (id: 21)
Các nút tương tự nhất:
Nút: iPhone 14 128GB  | Chính hãng VN/A (id=21)
    (iPhone 14 128GB  | Chính hãng VN/A) -[:Sản xuất bởi]-> (Apple)
    (iPhone 14 128GB  | Chính hãng VN/A) -[:Chipset]-> (Apple A15 Bionic 6 nhân)
    (iPhone 14 128GB  | Chính hãng VN/A) -[:CPU]-> ( 3.22 GHz)
    (iPhone 14 128GB  | Chính hãng VN/A) -[:Hệ điều hành]-> (iOS)
    (iPhone 14 128GB  | Chính hãng VN/A) -[:Phiên bản hệ điều hành]-> (iOS 16)
    (iPhone 14 128GB  | Chính hãng VN/A) -[:Kích thước màn hình]-> (6.1 inches)
    (iPhone 14 128GB  | Chính hãng VN/A) -[:Độ phân giải màn hình]-> (2532 x 1170 pixels)
    (iPhone 14 128GB  | Chính hãng VN/A) -[:Loại màn hình]-> (OLED)
    (iPhone 14 128GB  | Chính hãng VN/A) -[:Tần số quét]-> (60Hz)
    (iPhone 14 128GB  | Chính hãng VN/A) -[:Camera sau]-> (Camera góc rộng: 12MP, ƒ/1.5<br>Camera góc siêu rộng: 12MP, ƒ/2.4)
    (iPhone 1